# Introduction to Spatial Data Analysis

This notebook will introduce you to spatial data analysis using Python. There are several types of spatial data including Geotif, NetCDF, HDF5, GRIB. In this exercise, you will learn how to read, manipulate, plot and save NetCDF data. By the end of this exercise you should be able to:

1. Read NetCDF data using Xarray
2. Interrogate the data
3. Sub-set NetCDF data to a region and time of interest
4. Perform simple arithmetics on the data
5. Save the output as NetCDF data
6. Plot the results of the analysis

NetCDF (Network Common Data Form) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data (Ref: https://www.unidata.ucar.edu/software/netcdf/). It is also a community standard for sharing scientific data.

Data in netCDF format is:

* **Self-Describing:** A netCDF file includes information about the data it contains.
* **Portable:** A netCDF file can be accessed by computers with different ways of storing integers, characters, and floating-point numbers.
* **Scalable:** Small subsets of large datasets in various formats may be accessed efficiently through netCDF interfaces, even from remote servers.
* **Appendable:** Data may be appended to a properly structured netCDF file without copying the dataset or redefining its structure.
* **Sharable:** One writer and multiple readers may simultaneously access the same netCDF file.
* **Archivable:** Access to all earlier forms of netCDF data will be supported by current and future versions of the software.

There are several modules available for reading NetCDF data in Python and these include, *Netcdf4, Xarray, and Iris*. You can Explore the other packages in your own free time. Here we will make use of the **Xarray** module. Xarray (https://docs.xarray.dev/en/stable/index.html) makes working with labelled multi-dimensional arrays in Python simple, efficient, and fun! It depends on Numpy and Pandas modules.

To install xarray with its recommended dependencies using the conda command line tool:

* conda install -c conda-forge xarray dask netCDF4 bottleneck

You can also use pip to install xarray:

* python -m pip install xarray

In [ ]:
## Let's load the required module after installation

import xarray as xr                 ## Import xarray module into the Python environment and shorten the name as xr

The next step is to define the path to our dataset. The data is located in in our local folders. You can define either an absolute or relative path to your data in order to read the data using xarray. 

In [ ]:
## Let's define a variable path and assign 'https://drive.google.com/uc?export=download&id=' to it

path = '../../netcdf_data/'

Remember, in order to concatenante two strings you need to use the *'+'* sign. We will now use the *open_dataset()* function in the xarray module to read our NetCDF file that is saved in a Google Drive folder.

In [ ]:
## Reading a netcdf file saved in a Google drive folder defined by the strings stored in the variables path and nc_file.
## The two strings are concatenated using '+' sign.

ds = xr.open_dataset(path + 'cru_ts3.21.2011.2019.tmn.dat.nc')      ## Reading a netcdf file saved in a local folder.

In [ ]:
## Let's assess the class of our dataset which has been read into ds.

print('The class of ds is: ', type(ds))

We can interrogate our dataset further for us to fully understand the structure of data we are dealing with. Typing 'ds' in an empty cell should output something similar to the image below:

<img src="./ncdf_info2.png" width=900 height=400 />

In [ ]:
##  Let us interrogate our data. You can uncomment the other lines to view the output. 

ds                  ## Will show you more about the whole data
# ds.lon              ## Will show you more details about the longitude coordinate. Same can be done for the other coords (ds.lat, ds.time)
# ds.tmn              ## Shows more details about the data with variable name tmn.
# ds.stn              ## To get additional info about the 'stn' variable

In [ ]:
## Extracting the coordinate and variable values. You can uncomment the lines to view the output.

ds.lon.values          ## Extracts the values of longitude coordinate
# ds.tmn.values          ## Extracts the numerical values of tmn
# ds.stn.values          ## Extracts the values of stn

In [ ]:
## The attributes of each of the categories are stored as Dictionaries. Uncomment the lines to view the output.

type(ds.tmn.attrs)    ## Check the data type
# ds.tmn.attrs          ## List the available attributes of the variable 'tmn'

In [ ]:
## Let's extract the units of variable 'tmn'. Uncomment the lines to view the output.

ds.tmn.attrs.['units']      ## Show the units of variable 'tmn'
# ds.tmn.attrs['long_name']   ## Show the long name of variable 'tmn'

## Sub-setting Xarray Dataset

We have seen that the NetCDF data covers global land areas from the Climate Research Unit (CRU). The variable of interest is near surface temperature minimum (tmn). We can sub-set the data to a region of interest, for us the region is the East Africa/ the Greater Horn of Africa region. It is also possible to select a point of interest. It is possible to use other sophisticated methods such as using a shapefile to sub-set the data. We are hower going to concentrate on simple method of defining the latitude and longitude bounding box of our region of interest.

Let's use the following coodinate of the GHA bounding box to sub-set our global data to our region:

* Latitude: -15.0 degreesN to 24.0 degreesN
* Longitude: 21.0 degreesE to 52.0 degreesE

In [ ]:
## Subsettting the global data to GHA region

ds_gha = ds.sel(lat=slice(-15.0, 24.0), lon=slice(21.0, 52.0))   ## Sub-set the global data to GHA region using bounding box
# ds_stn = ds.sel(lat=2.3, lon=35.9, method='nearest')             ## This will extract for a single point.

In [ ]:
## Ucomment the line to interrogate the data.

ds_gha           ## This will show that the dimensions have reduced to (60, 78, 108) from (720, 360, 108)

## Simple Spatial Summaries

The data is now sub-set for our region of interest. Next, we will perform some simple arithmetics on our spatial data to summarize the data for our region of interest. Sophisticated analysis can be done, however, we will only concentrate on the simple methods to demonstrate the concepts. We will concentrate on the following:

* Monthly climatology of GHA
* Seasonal climatology
* Seasonal Anomalies

### Monthly Climatology

The climatology is given by the sum of the data points divided by the number of points/observations. This is given by the equation below. We want to implement this for each month in a year. We will make use of xarray in built math functionalities for this task.

$$
\bar{X} = \frac{1}{N} \sum \limits_{i=1}^N X_{i}
$$

Monthly mean analysis requires us to group our data according to months. The number of groups will be 12 corresponding to the number of months in a year. The mean of each month can therefore be obtain after grouping the data. We will usw the **groupby()** function in xarray.

In [ ]:
## Let's group our data according to months

ds_mon = ds_gha.groupby('time.month');ds_mon       ## Grouping the data by months

In [ ]:
## We will find the mean of the grouped data

ds_clim = ds_mon.mean('time');ds_clim              ## Mothly climatology of tmin over our area of interes, GHA.

In [ ]:
## Note the above two steps can be achieved using a single command. Uncomment to find out!

ds_clim2 = ds_gha.groupby('time.month').mean('time');ds_clim2  ## The above two steps using one line

### Seasonal Climatology

The formular for calculating seasonal climatology is similar to that of simple mean. The only difference is that it will be applied over a particular season of interest. There are two approaches to do this:

1. First group our dataset according to seasons ['DJF', 'JJA', 'MAM', 'SON'] as defined in xarray
2. Select the months of interest that define the particular season of interest from the data, then calculate the mean.

We will use the first approach. The second approach is left as an exercise.

In [ ]:
## Group data according to predefined seasons in xarray

ds_seas = ds_gha.groupby('time.season');ds_seas      ## Group data according to seasons

In [ ]:
## Calculate the climatology of the seasons

seas_clim = ds_seas['MAM'].mean('time');seas_clim    ## Calculate the climatology mean for MAM sea

In [ ]:
## To do this conviniently using one line of code

seas_clim2 = ds_gha.groupby('time.season')['MAM'].mean('time');seas_clim2   ## Convenient way of achieving the above using one line of code

In [ ]:
## Let's view the data

seas_clim2.tmn.plot()

### Seasonal Mean Anomalies

The formula for calculating the mean seasonal anomaly is:

$$
\hat{X} = X_{i} - \bar{X}
$$

Where $\bar{X}$ and $X_{i}$ are the climatology and observation for the particular season of interest respectively. 

In [ ]:
## Group data according to predefined seasons in xarray

ds_seas2 = ds_gha.groupby('time.season');ds_seas2      ## Group data according to seasons

In [ ]:
## Calculate the climatology of the seasons

seas_clim2 = ds_seas2['MAM'].mean('time');seas_clim2    ## Calculate the climatology mean of MAM season

In [ ]:
## Calculate the yearly anomaly

ds_anom = ds_seas2['MAM'] - seas_clim2

In [ ]:
## Let's view the data

ds_anom.tmn[0,:,:].plot()

In [ ]:
## Let's view the data

ds_anom.tmn[0,:,:].plot()

## Writing out netcdf files

In [ ]:
## The analysed data can be written out as a netcdf file.
## Save the seasonal anomalies output file to a directory. 
## The function to use is "Dataset.to_netcdf()"

ds_anom.to_netcdf('./gha_seasonal_anomalies.nc', mode='w')

## Plotting the Results

We will be using the most popular plotting modules in Python called Matplotlib to plot or results. Most of the Matplotlib functionality is available in the pyplot submodule, in most cases this is imported and shortened as plt.

Matplotlib has two fundamental objects: *fig* and *axis*.

* Fig object defines the main container (canvas/plot area) for plotting. It can contain multiple plots inside it.
* Axis object defines an individual plot/graph and can contain one or more axes.

In [ ]:
## Import the plotting module

import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1,1)                             ## define the fig object to contain one axis
plt.contourf(ds_anom.lon, ds_anom.lat, ds_anom.tmn[0,:,:], cmap='RdBu')  ## plotting filled contours
plt.title(ds_anom.time[0].values, fontsize=14)          ## Add plot title which is the first time step
plt.xlabel('Longitude', fontsize=14)
plt.ylabel('Latitude', fontsize=14)
plt.colorbar().set_label('Temp (degC)', fontsize=14)    ## Add colorbar and label
# plt.show()

### Complex Plots

It is possible to further customize the plots to your needs. In this exercise we will learn how to add additional details to our plot. We will also add multiple plots, add country border in one figure and save the output.

In [ ]:
## Let's load the necessary modules

import numpy as np      ## Numerical Python library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  ## The crs module defines Coordinate Reference Systems and the transformations
#  between them
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER ## used to determine draw time behaviour of the gridlines and labels
import cartopy.feature as cfeat   ## Cartographic features from cartopy

In [ ]:
fig = plt.figure(figsize=(7, 5))      ## Define the main container for all figures

proj = ccrs.PlateCarree(central_longitude=0)  ## Define the coordinate reference system
ax = plt.subplot(111, projection=proj)   ## Divide our figure into four and use the coordinate reference system defined by proj

## Add catographic features to the map using cartopy

ax.add_feature(cfeat.BORDERS, linewidth=1., edgecolor='black')  ## linestyle=':',
ax.add_feature(cfeat.LAKES,facecolor='none', edgecolor='black',linewidth=0.8)
ax.add_feature(cfeat.COASTLINE,linewidth=1.)

## Use filled contour from matplotlib to plot our data

cs = plt.contourf(ds_anom.lon, ds_anom.lat, ds_anom.tmn[0,:,:], levels=np.arange(-5.5, 6, 1), 
                  cmap=plt.cm.RdBu.reversed(), extend = 'both')

## Add more details
ax.set_extent([21, 52, -15, 23.5])     ## Set extent of our data
ax.set_xticks(np.arange(21, 52, 10))   ## Set the x-ticks marks at certain intervals (accepts integers only)
ax.set_yticks(np.arange(-13, 23, 5))   ## Set the y-ticks marks at certain intervals (accepts integers only)

## Add the degree symbol and the direction (north, south, east and west)
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER)
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)

## Define the axis labels
labelsx = ax.get_xticklabels()   ## Get the xticks labels from the (ax.set_xticks)  
labelsy = ax.get_yticklabels()   ## Get the yticks labels from the (ax.set_yticks) 
plt.setp(labelsx, fontsize=10)   ## Add the labels to the plot and set the font size
plt.setp(labelsy, fontsize=10)   ## Add the labels to the plot and set the font size
ax.label_outer()                 ## Automatically switch off the axis labels for multipanel plots

## Add lebels to each of our multipanel plots
ax.text(0.9, 0.90, '(a)', fontsize = 12, horizontalalignment='center',
        verticalalignment='center', rotation='horizontal', transform=ax.transAxes)
plt.colorbar(cs, orientation='horizontal').set_label('Rainfall (mm)', fontsize=14)
# plt.show()

### Practice Exercise

1. Plot a multipanel figure containing four plots of the seasonal anomalies (DJF, MAM, JJA, SON).
2. Label the four panels ('a', 'b', 'c', 'd')
3. Add plot titles (i.e., season names)
4. Save the figure.